In [2]:
# importation des packages néccessaire
print("[INFO] Chargement des packages...")
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
# from imutils.video import VideoStream
import face_recognition
import numpy as np
import imutils
import time
import cv2

print("[INFO] chargement du modèle...")
model = load_model("mask_detector.model")

# initialisation de la lecture en direct
print("[INFO] Démarage de la webcam...")
cap = cv2.VideoCapture(0) 
while True:
    faces = []
    locs = []
    preds = []
    # capturer l'image et s'assurer que la largueur est de 400pixels
    sucess, image = cap.read()
    image = imutils.resize(image, width=400)

    # appel de la fonction detect_and_predict_mask
    detections = face_recognition.face_locations(image)
    for faceLoc in detections:
        startY,endX, endY, startX = faceLoc

        # vérifier le cadre ne dépasse pas la photo
#         (startX, startY) = (max(0, startX), max(0, startY))
#         (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

        # extraire le visage, convertir en rgb , redimensionner, faire le préprocessing
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)

        # ajouter le visage et la localisation dans une liste
        faces.append(face)
        locs.append((startX, startY, endX, endY))

        # si au moins une face est détecté
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = model.predict(faces, batch_size=32)
#     (locs, preds) = detect_and_predict_mask(image, model)

    # recuper la localisation et la prédiction
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # détecter la classe prédicte et la couleur liée
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # inclus la probabilité sous forme de text
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # dessiner un rectangle et mettre un texte
        cv2.putText(image, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

    # afficher
    cv2.imshow("Frame", image)

    # si la touche "q" est appuyée arreter la boucle infini
    if  (cv2.waitKey(1) & 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()

[INFO] Chargement des packages...
[INFO] chargement du modèle...
[INFO] Démarage de la webcam...
